# Prepare data for Pipeline

## Setup and settings

In [1]:
# Importing packages
import os
import yaml
import logging
import pandas as pd
from Bio.Seq import Seq
from qiime2 import Artifact, Visualization, Metadata
from qiime2.plugins import demux
from qiime2.plugins.cutadapt.methods import trim_paired, trim_single
from qiime2.plugins.demux.methods import filter_samples

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [2]:
params_path = os.path.join('..', 'params', 'parameter-set-01.yaml')
experiment_name = 'exp-01'
base_dir = os.path.join('/', 'home', 'username', 'pipeline-dir')
manifest_file = os.path.join(base_dir, 'data', 'raw', 'manifest', 'manifest.csv')
replace_files = False
trim = None
metadata_file = None
threads = 1
demux_file = None

In [3]:
# Parameters
experiment_name = "ana-flavia-NCxNR-trim"
base_dir = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri"
manifest_file = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/manifest/manifest-ana-flavia-NCxNR.csv"
metadata_file = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/metadata/metadata-ana-flavia-NCxNR.tsv"
class_col = "group-id"
classifier_file = "/home/lauro/nupeb/rede-micro/models/silva-138-99-nb-classifier.qza"
top_n = 20
replace_files = False
phred = 20
trunc_f = 0
trunc_r = 0
overlap = 12
threads = 6
trim = {
    "overlap": 8,
    "forward_primer": "CCTACGGGRSGCAGCAG",
    "reverse_primer": "GGACTACHVGGGTWTCTAAT",
}


In [4]:
out_dir = os.path.join(base_dir, 'experiments', experiment_name, 'qiime-artifacts')
img_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name, 'imgs'))
data_dir =  os.path.abspath(os.path.join(base_dir, 'data'))
raw_data_dir =  os.path.abspath(os.path.join(data_dir, 'raw'))
interim_data_dir =  os.path.abspath(os.path.join(data_dir, 'interim'))

### Defining names and paths

In [5]:
# Create path if it not exists
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)
    print(f'New qiime-artifacts folder path created: {out_dir}')

# Create path if it not exists
if not os.path.isdir(img_folder):
    os.makedirs(img_folder)
    print(f'New img folder path created: {img_folder}')

# Define the output artifact full path
if demux_file is None:
    demux_file = os.path.join(out_dir, 'demux-paired.qza')
demux_view = os.path.join(out_dir, 'demux-paired.qzv')
demux_file_trim = os.path.join(out_dir, 'demux-paired-trim.qza')
demux_view_trim = os.path.join(out_dir, 'demux-paired-trim.qzv')

New qiime-artifacts folder path created: /home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-NCxNR-trim/qiime-artifacts
New img folder path created: /home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-NCxNR-trim/imgs


## Step execution

This Step import all `fastq` files in a **QIIME2 Artifact** object and save it to a new `qza` file of `SampleData[PairedEndSequencesWithQuality]`

In [6]:
manifest_df = pd.read_csv(manifest_file)
n_directions = len(manifest_df['direction'].unique())
if n_directions == 1:
    d_type = 'SampleData[SequencesWithQuality]'
    v_type = 'SingleEndFastqManifestPhred33'
elif n_directions == 2:
    d_type = 'SampleData[PairedEndSequencesWithQuality]'
    v_type = 'PairedEndFastqManifestPhred33'
else:
    print(f'ERROR: invalid number of directions {n_directions}')

In [7]:
# If the metadata file is not defined, use the default metadata file
metadata_qa = Metadata.load(metadata_file)

In [8]:
# If the demux file does not exist or if the old file will be replaced
if not os.path.isfile(demux_file) or replace_files:
    interim_demux_path = os.path.join(interim_data_dir, "demux-paired.qza")
    # Load Artifact from interim folder
    if os.path.isfile(interim_demux_path):
        artifact = Artifact.load(interim_demux_path)
    # Create new Artifact using Manifest
    else:
        artifact = Artifact.import_data(d_type, manifest_file, view_type=v_type)
        if not os.path.isfile(interim_demux_path):
            artifact.save(interim_demux_path)
    # Filter and Save Artifact
    artifact = filter_samples(demux=artifact, metadata=metadata_qa).filtered_demux
    artifact.save(demux_file)
    # Visualization
    if os.path.isfile(demux_view):
        demux_view_obj = Visualization.load(demux_view)
    else:
        demux_view_obj = demux.visualizers.summarize(artifact).visualization
        Visualization.save(demux_view_obj, filepath=demux_view)
elif os.path.isfile(demux_file):
    # Load Artifact
    artifact = Artifact.load(demux_file)
    artifact = filter_samples(demux=artifact, metadata=metadata_qa).filtered_demux
    # Visualization
    if os.path.isfile(demux_view):
        demux_view_obj = Visualization.load(demux_view)
    else:
        demux_view_obj = demux.visualizers.summarize(artifact).visualization
        Visualization.save(demux_view_obj, filepath=demux_view)

## Step report


In [9]:
print(demux_view_obj)

<visualization: Visualization uuid: e0d2ff17-7f5b-404f-a4fe-5f1ecd878f3b>


In [10]:
# Render Visualization
demux_view_obj

<visualization: Visualization uuid: e0d2ff17-7f5b-404f-a4fe-5f1ecd878f3b>

[cutadapt](https://docs.qiime2.org/2022.2/plugins/available/cutadapt/trim-paired/)

We trim the forward primer and the reverse complement of the reverse primer from the forward reads. We trim the reverse primer and reverse complement of the forward primer from the reverse reads.

In [11]:
demux_file_trim = os.path.join(out_dir, 'demux-paired-trim.qza')
demux_view_trim = os.path.join(out_dir, 'demux-paired-trim.qzv')

if trim and (not os.path.isfile(demux_file_trim) or replace_files):
    forward_primer = [trim['forward_primer']] # ['CCTACGGGRSGCAGCAG']
    reverse_primer = [trim['reverse_primer']] # ['GGACTACHVGGGTWTCTAAT']
    forward_reverse_complement = [str(Seq(forward_primer[0]).reverse_complement())]
    reverse_reverse_complement = [str(Seq(reverse_primer[0]).reverse_complement())]

    res = None
    
    if n_directions == 1:
        res = trim_single(
            demultiplexed_sequences=artifact,
            front=forward_primer,
            adapter=reverse_reverse_complement,
            cores=threads,
            indels=False,
            match_read_wildcards=True,
            match_adapter_wildcards=True,
            error_rate=0.01,
            discard_untrimmed=True,
        ).trimmed_sequences
    elif n_directions == 2:
        res = trim_paired(
            demultiplexed_sequences=artifact,
            front_f=forward_primer,
            front_r=reverse_primer,
            # adapter_f=reverse_reverse_complement,
            # adapter_r=forward_reverse_complement,
            cores=threads,
            overlap=trim['overlap'],
            indels=False,
            match_read_wildcards=True,
            match_adapter_wildcards=True,
            error_rate=0.01,
            discard_untrimmed=True,
        ).trimmed_sequences
    else:
        print(f'ERROR: invalid number of directions {n_directions}')

    if res is not None:
        res.save(demux_file_trim)
        Visualization.save(demux.visualizers.summarize(res).visualization, filepath=demux_view_trim)

Running external command line application. This may print messages to stdout and/or stderr.
The commands to be run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: cutadapt --cores 6 --error-rate 0.01 --times 1 --overlap 8 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-m94t6ggh/S210421121673_0_L001_R1_001.fastq.gz -p /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-m94t6ggh/S210421121673_1_L001_R2_001.fastq.gz --front CCTACGGGRSGCAGCAG -G GGACTACHVGGGTWTCTAAT --no-indels --match-read-wildcards --discard-untrimmed /tmp/qiime2-archive-rf1lscat/cef1f2a8-bac6-49b1-b295-0e47d8e89e18/data/S210421121673_0_L001_R1_001.fastq.gz /tmp/qiime2-archive-rf1lscat/cef1f2a8-bac6-49b1-b295-0e47d8e89e18/data/S210421121673_1_L001_R2_001.fastq.gz



This is cutadapt 3.5 with Python 3.8.12
Command line parameters: --cores 6 --error-rate 0.01 --times 1 --overlap 8 --minimum-length 1 -o /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-m94t6ggh/S210421121673_0_L001_R1_001.fastq.gz -p /tmp/q2-CasavaOneEightSingleLanePerSampleDirFmt-m94t6ggh/S210421121673_1_L001_R2_001.fastq.gz --front CCTACGGGRSGCAGCAG -G GGACTACHVGGGTWTCTAAT --no-indels --match-read-wildcards --discard-untrimmed /tmp/qiime2-archive-rf1lscat/cef1f2a8-bac6-49b1-b295-0e47d8e89e18/data/S210421121673_0_L001_R1_001.fastq.gz /tmp/qiime2-archive-rf1lscat/cef1f2a8-bac6-49b1-b295-0e47d8e89e18/data/S210421121673_1_L001_R2_001.fastq.gz
Processing reads on 6 cores in paired-end mode ...


Finished in 3.78 s (10 µs/read; 5.87 M reads/minute).

=== Summary ===

Total read pairs processed:            370,042
  Read 1 with adapter:                 359,997 (97.3%)
  Read 2 with adapter:                 343,009 (92.7%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           36,262 (9.8%)
Pairs written (passing filters):       333,780 (90.2%)

Total basepairs processed:   188,721,420 bp
  Read 1:   112,862,810 bp
  Read 2:    75,858,610 bp
Total written (filtered):    155,609,062 bp (82.5%)
  Read 1:    94,978,565 bp
  Read 2:    60,630,497 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 359997 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	3	5.6	0	3
9	4	1.4	0	4
10	3	0.4	0	3
11	2	0.1	0	2
12	4	0.0	0	4
13	4	0.0	0	4
14	12	0.0	0	12
15	9	0.0	0	9
16	19	0.0	0	19
17	109	0.0	0	109
1

Finished in 5.08 s (9 µs/read; 6.61 M reads/minute).

=== Summary ===

Total read pairs processed:            559,807
  Read 1 with adapter:                 544,656 (97.3%)
  Read 2 with adapter:                 519,894 (92.9%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           53,686 (9.6%)
Pairs written (passing filters):       506,121 (90.4%)

Total basepairs processed:   285,501,570 bp
  Read 1:   170,741,135 bp
  Read 2:   114,760,435 bp
Total written (filtered):    235,957,214 bp (82.6%)
  Read 1:   144,019,667 bp
  Read 2:    91,937,547 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 544656 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	5	8.5	0	5
9	4	2.1	0	4
10	3	0.5	0	3
11	7	0.1	0	7
12	7	0.0	0	7
13	6	0.0	0	6
14	17	0.0	0	17
15	16	0.0	0	16
16	22	0.0	0	22
17	155	0.0	0	155


Finished in 3.33 s (10 µs/read; 6.23 M reads/minute).

=== Summary ===

Total read pairs processed:            345,762
  Read 1 with adapter:                 336,819 (97.4%)
  Read 2 with adapter:                 321,022 (92.8%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           32,941 (9.5%)
Pairs written (passing filters):       312,821 (90.5%)

Total basepairs processed:   176,338,620 bp
  Read 1:   105,457,410 bp
  Read 2:    70,881,210 bp
Total written (filtered):    145,841,282 bp (82.7%)
  Read 1:    89,015,404 bp
  Read 2:    56,825,878 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 336819 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	5	5.3	0	5
9	3	1.3	0	3
10	5	0.3	0	5
11	2	0.1	0	2
12	4	0.0	0	4
13	5	0.0	0	5
14	12	0.0	0	12
15	9	0.0	0	9
16	11	0.0	0	11
17	92	0.0	0	92
18	

...

Finished in 3.44 s (9 µs/read; 6.38 M reads/minute).

=== Summary ===

Total read pairs processed:            365,427
  Read 1 with adapter:                 355,998 (97.4%)
  Read 2 with adapter:                 339,330 (92.9%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           34,713 (9.5%)
Pairs written (passing filters):       330,714 (90.5%)

Total basepairs processed:   186,367,770 bp
  Read 1:   111,455,235 bp
  Read 2:    74,912,535 bp
Total written (filtered):    154,181,570 bp (82.7%)
  Read 1:    94,107,785 bp
  Read 2:    60,073,785 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 355998 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	1	5.6	0	1
9	2	1.4	0	2
10	4	0.3	0	4
11	4	0.1	0	4
12	3	0.0	0	3
13	1	0.0	0	1
14	19	0.0	0	19
15	9	0.0	0	9
16	25	0.0	0	25
17	99	0.0	0	99
18	5

Finished in 1.02 s (11 µs/read; 5.63 M reads/minute).

=== Summary ===

Total read pairs processed:             95,852
  Read 1 with adapter:                  93,250 (97.3%)
  Read 2 with adapter:                  88,539 (92.4%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:            9,674 (10.1%)
Pairs written (passing filters):        86,178 (89.9%)

Total basepairs processed:    48,884,520 bp
  Read 1:    29,234,860 bp
  Read 2:    19,649,660 bp
Total written (filtered):     40,178,039 bp (82.2%)
  Read 1:    24,522,719 bp
  Read 2:    15,655,320 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 93250 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
9	5	0.4	0	5
10	1	0.1	0	1
11	2	0.0	0	2
12	5	0.0	0	5
13	2	0.0	0	2
14	1	0.0	0	1
15	3	0.0	0	3
16	2	0.0	0	2
17	24	0.0	0	24
18	14644	0.0	0	1464

Finished in 3.08 s (10 µs/read; 6.18 M reads/minute).

=== Summary ===

Total read pairs processed:            316,849
  Read 1 with adapter:                 308,351 (97.3%)
  Read 2 with adapter:                 292,785 (92.4%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           31,765 (10.0%)
Pairs written (passing filters):       285,084 (90.0%)

Total basepairs processed:   161,592,990 bp
  Read 1:    96,638,945 bp
  Read 2:    64,954,045 bp
Total written (filtered):    132,905,965 bp (82.2%)
  Read 1:    81,120,487 bp
  Read 2:    51,785,478 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 308351 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	4	4.8	0	4
9	1	1.2	0	1
10	1	0.3	0	1
11	9	0.1	0	9
12	4	0.0	0	4
13	5	0.0	0	5
14	17	0.0	0	17
15	20	0.0	0	20
16	21	0.0	0	21
17	83	0.0	0	83


Finished in 2.51 s (10 µs/read; 5.93 M reads/minute).

=== Summary ===

Total read pairs processed:            248,647
  Read 1 with adapter:                 241,815 (97.3%)
  Read 2 with adapter:                 229,975 (92.5%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           24,848 (10.0%)
Pairs written (passing filters):       223,799 (90.0%)

Total basepairs processed:   126,809,970 bp
  Read 1:    75,837,335 bp
  Read 2:    50,972,635 bp
Total written (filtered):    104,332,265 bp (82.3%)
  Read 1:    63,680,774 bp
  Read 2:    40,651,491 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 241815 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	3	3.8	0	3
9	1	0.9	0	1
10	4	0.2	0	4
12	5	0.0	0	5
13	1	0.0	0	1
14	8	0.0	0	8
15	10	0.0	0	10
16	6	0.0	0	6
17	69	0.0	0	69
18	37474	0.0	0	37

Finished in 2.05 s (10 µs/read; 6.07 M reads/minute).

=== Summary ===

Total read pairs processed:            207,454
  Read 1 with adapter:                 201,975 (97.4%)
  Read 2 with adapter:                 192,058 (92.6%)

== Read fate breakdown ==
Pairs that were too short:                   0 (0.0%)
Pairs discarded as untrimmed:           20,423 (9.8%)
Pairs written (passing filters):       187,031 (90.2%)

Total basepairs processed:   105,801,540 bp
  Read 1:    63,273,470 bp
  Read 2:    42,528,070 bp
Total written (filtered):     87,193,544 bp (82.4%)
  Read 1:    53,220,325 bp
  Read 2:    33,973,219 bp

=== First read: Adapter 1 ===

Sequence: CCTACGGGRSGCAGCAG; Type: regular 5'; Length: 17; Trimmed: 201975 times

Minimum overlap: 8
No. of allowed errors:
1-17 bp: 0

Overview of removed sequences
length	count	expect	max.err	error counts
8	3	3.2	0	3
9	3	0.8	0	3
10	2	0.2	0	2
11	3	0.0	0	3
12	2	0.0	0	2
13	2	0.0	0	2
14	11	0.0	0	11
15	7	0.0	0	7
16	8	0.0	0	8
17	60	0.0	0	60
18	31